In [4]:
import numpy as np
import math
import matplotlib.pyplot as plt
import random

In [5]:
def forwardX(L1,L2,shoulder,elbow):
    return L1*math.cos(shoulder) + L2*math.cos(shoulder+elbow)
def forwardY(L1,L2,shoulder,elbow):
    return L1*math.sin(shoulder) + L2*math.sin(shoulder+elbow)
def forward(L1,L2,shoulder,elbow):
    print('Horizonal:', forwardX(L1,L2,shoulder,elbow))
    print('Vertical:', forwardY(L1,L2,shoulder,elbow))

In [6]:
def bisection(L1, L2, x, y, a, b, d, e, TOL):
    print('Bisection')
    if L1 <= 0 or L2 <= 0:
        print('Please ensure both of your bone lengths are greater than 0.')
        return

    if np.abs(L1 - L2) > math.sqrt(x**2 + y**2) or math.sqrt(x**2 + y**2) > L1 + L2:
        print('Sorry, the arm cannot reach the point you provided.')
        return
    
    fa = (x - L1*math.cos(a))**2 + (y - L1*math.sin(a))**2 - L2**2
    fb = (x - L1*math.cos(b))**2 + (y - L1*math.sin(b))**2 - L2**2
            
    if np.sign(fa)*np.sign(fb) > 0:
        print('f(a)f(b)<0 not satisfied') 
        return # stop execution
    n=1            
    while np.abs(a-b)>TOL:
        c = (a+b)/2
        fc = (x - L1*math.cos(c))**2 + (y - L1*math.sin(c))**2 - L2**2 
        n=n+1
        #if np.isclose(f(c), 0):
        #    print('Approximate  root', c, 'has been obtained in', n, 'steps')
        #    return
        if np.sign(fc)*np.sign(fa)<0:
            b = c
            fb=fc
        else:
            a = c
            fa= fc
        #print(a, ', ', b)
        #print(fa, ', ', fb)
    c=(a+b)/2

    shoulder = c%(2*math.pi)
    print('Shoulder interval: [', np.degrees(a), np.degrees(b), ']')
    print('Approximate  angle', np.degrees(shoulder),' degrees has been obtained in', n, 'steps')

    gd = L1*math.sin(shoulder) + L2*math.sin(shoulder + d) - y
    ge = L1*math.sin(shoulder) + L2*math.sin(shoulder + e) - y

    if np.sign(gd)*np.sign(ge) > 0:
        print('g(d)g(e)<0 not satisfied') 
        return # stop execution
    m=1            
    while np.abs(d-e)>TOL:
        c = (d+e)/2
        gc = L1*math.sin(shoulder) + L2*math.sin(shoulder + c) - y   
        m=m+1
        #if np.isclose(g(c), 0):
        #    print('Approximate  root', c, 'has been obtained in', n, 'steps')
        #    return
        if np.sign(gc)*np.sign(gd)<0:
            e = c
            ge=gc
        else:
            d = c
            gd= gc
        #print(d, ', ', e)
        #print(gd, ', ', ge)    
        
    c=(d+e)/2

    elbow = c%(2*math.pi)

        
    print('Elbow interval: [', np.degrees(d), np.degrees(e), ']')
    print('Approximate  angle', np.degrees(elbow),' degrees has been obtained in', m, 'steps')
    if abs(L2*math.sin(elbow)/L1) <= 1:
        print('Elbow-down configuation: Shoulder at ', math.degrees(shoulder + math.asin(L2*math.sin(elbow)/L1))%360, ' degrees, and elbow at ', (180+math.degrees(elbow))%360,' degrees')
    else:
        print('Elbow-down configuation could not be found.')

bisection(1,1,-0.0481988113872,1.64996738863,.5,2,1,2,1e-40)

Bisection
Shoulder interval: [ 57.29577951320702 57.29577951320702 ]
Approximate  angle 57.29577951320702  degrees has been obtained in 55 steps
Elbow interval: [ 68.7549354154441 68.7549354154441 ]
Approximate  angle 68.7549354154441  degrees has been obtained in 54 steps
Elbow-down configuation: Shoulder at  126.05071492865112  degrees, and elbow at  248.7549354154441  degrees


In [7]:
def newtonMethod(f,Df,x0,TOL,N):
    '''Approximate solution of f(x)=0 by Newton's method.
    Input
    ----------
    f : function
    Df : derivative of f(x).
    x0 :initial guess  
    TOL1: stopping criteria 
    N:  number of iterations 
    '''
    
    xn = x0
    for n in range(0,N):
        fxn = f(xn)
        Dfxn = Df(xn)
        if Dfxn == 0:
            print('Zero derivative. No solution found.')
            return None
        xn = xn - fxn/Dfxn
        if abs(fxn/Dfxn) < TOL:
            xn = xn%(2*math.pi)
            #print('Approximate solution', xn, 'after steps',n)
            return xn
    print('Exceeded maximum iterations. No solution found.')
    return None

def newtonSolution(L1,L2,x,y,b,TOL,N):
    print('Newton')
    if L1 <= 0 or L2 <= 0:
        print('Please ensure both of your bone lengths are greater than 0.')
        return

    if np.abs(L1 - L2) > math.sqrt(x**2 + y**2) or math.sqrt(x**2 + y**2) > L1 + L2:
        print('Sorry, the arm cannot reach the point you provided.')
        return

    def F(a):
        return (x - L1*math.cos(a))**2 + (y - L1*math.sin(a))**2 - L2**2
    def DF(a):
        return 2*(x - L1*math.cos(a))*L1*math.sin(a) - 2*(y - L1*math.sin(a))*L1*math.cos(a)
    shoulder = newtonMethod(F,DF,b,TOL,N)
    
    def G(a):
        return L1*math.sin(shoulder) + L2*math.sin(shoulder + a) - y
    def DG(a):
        return L2*math.cos(newtonMethod(F,DF,b,TOL,N) + a)
    elbow = newtonMethod(G,DG,b,TOL,N)
    
    print('Approximate angle of shoulder:',math.degrees(shoulder),'degrees')
    print('Approximate angle of elbow:',math.degrees(elbow),'degrees')
    if abs(L2*math.sin(elbow)/L1) <= 1:
        print('Elbow-down configuation: Shoulder at ', math.degrees(shoulder + math.asin(L2*math.sin(elbow)/L1))%360, ' degrees, and elbow at ', (180+math.degrees(elbow))%360,' degrees')
    else:
        print('Elbow-down configuation could not be found.')


newtonSolution(1,1,-0.0481988113872,1.64996738863,.8,1e-15,100)

Newton
Approximate angle of shoulder: 57.295779513207016 degrees
Approximate angle of elbow: 68.75493541544388 degrees
Elbow-down configuation: Shoulder at  126.05071492865089  degrees, and elbow at  248.75493541544387  degrees


In [8]:
def g(L1,L2,x,y,a):
    return (x - L1*math.cos(a))**2 + (y - L1*math.sin(a))**2 - L2**2 + a

def h(L1,L2,y,a,b):
    return L1*math.sin(a) + L2*math.sin(a + b) - y + b

def fixedp(L1,L2,x,y,a0,b0,TOL,N):
    print('Fixed-point')
    if L1 <= 0 or L2 <= 0:
        print('Please ensure both of your bone lengths are greater than 0.')
        return

    if np.abs(L1 - L2) > math.sqrt(x**2 + y**2) or math.sqrt(x**2 + y**2) > L1 + L2:
        print('Sorry, the arm cannot reach the point you provided.')
        return
    
    e = 1
    i = 0
    ai = []
    while(e > TOL and i < N):
        a=g(L1,L2,x,y,a0)      # fixed point equation
        e= np.abs(a0-a) # error at the current step
        a0=a
        ai.append(a0)  # save the solution of the current step
        #print(ai,i)
        i= i + 1
    if e< TOL:
        print('Approximate angle of shoulder:', np.degrees(a0)%360,' degrees') 
    else:
        print('The first fixed point iteration diverges') 
        return(ai,i)
    
    e = 1
    i = 0
    bi = []
    while(e > TOL and i < N):
        b=h(L1,L2,y,a0,b0)      # fixed point equation
        e= np.abs(b0-b) # error at the current step
        b0=b
        bi.append(b0)  # save the solution of the current step
        #print(ai,i)
        i= i + 1
    if e< TOL:
        print('Approximate angle of elbow:', np.degrees(b0)%360,' degrees')
        if abs(L2*math.sin(b0)/L1) <= 1:
            print('Elbow-down configuation: Shoulder at ', math.degrees(a0 + math.asin(L2*math.sin(b0)/L1))%360, ' degrees, and elbow at ', (180+math.degrees(b0))%360,' degrees')
        else:
            print('Elbow-down configuation could not be found.')
    else:
        print('The second fixed point iteration diverges') 
        return(ai,i)

fixedp(1,1,-0.0481988113872,1.64996738863,.5,1,1e-5,100)

Fixed-point
Approximate angle of shoulder: 57.29602917005287  degrees
Approximate angle of elbow: 68.75452866156621  degrees
Elbow-down configuation: Shoulder at  126.05055783161909  degrees, and elbow at  248.7545286615662  degrees


In [50]:
def threeMethods(L1,L2,x,y):
    guess = random.uniform(0,2*math.pi)
    guess2 = random.uniform(0,2*math.pi)
    bisection(L1,L2,x,y,(1e-10)*random.randint(0,1),guess,(1e-10)*random.randint(0,1),guess2,1e-5)
    print('')
    fixedp(L1,L2,x,y,guess,guess2,1e-5,10000)
    print('')
    newtonSolution(L1,L2,x,y,guess,1e-5,1000)

In [10]:
threeMethods(1,1,1,.8)

Bisection
f(a)f(b)<0 not satisfied

Fixed-point
Approximate angle of shoulder: 348.47465025330115  degrees
Approximate angle of elbow: 102.69454671043167  degrees
Elbow-down configuation: Shoulder at  65.78010354286944  degrees, and elbow at  282.69454671043167  degrees

Newton
Approximate angle of shoulder: 348.4749283513514 degrees
Approximate angle of elbow: 100.36975979076492 degrees
Elbow-down configuation: Shoulder at  68.10516856058649  degrees, and elbow at  280.36975979076493  degrees


In [11]:
def randomThreeMethods():
    L1 = random.uniform(0,10)
    L2 = random.uniform(0,10)
    x = random.uniform(0, L1+L2)
    y = random.uniform(math.sqrt(max(0,(L1-L2)**2-x**2)),math.sqrt((L1+L2)**2-x**2))
    x = x*(-1)**random.randint(1,2)
    y = y*(-1)**random.randint(1,2)

    print('Upper arm length:', L1)
    print('Forearm length:', L2)
    print('Horizontal coordinate:', x)
    print('Vertical coordinatate:', y)
    threeMethods(L1,L2,x,y)

randomThreeMethods()

Upper arm length: 4.098521804828019
Forearm length: 6.532137786070984
Horizontal coordinate: -8.025823390327451
Vertical coordinatate: 3.3572317952628112
Bisection
Shoulder interval: [ 111.60950828897889 111.61003851049516 ]
Approximate  angle 111.60977339973702  degrees has been obtained in 19 steps
Elbow interval: [ 72.3687694758815 72.36905923376311 ]
Approximate  angle 72.36891435482231  degrees has been obtained in 20 steps
Elbow-down configuation could not be found.

Fixed-point
The first fixed point iteration diverges

Newton
Approximate angle of shoulder: 111.60993392367061 degrees
Approximate angle of elbow: 244.4115477930594 degrees
Elbow-down configuation could not be found.


In [86]:
def checkMethods(L1,L2,shoulder,elbow):
    print('Upper arm length:',L1)
    print('Lower arm length:',L2)
    print('Shoulder:', math.degrees(shoulder))
    print('Elbow',math.degrees(elbow))
    forward(L1,L2,shoulder,elbow)
    threeMethods(L1,L2,forwardX(L1,L2,shoulder,elbow),forwardY(L1,L2,shoulder,elbow))
    print('')
    
checkMethods(1,1,.5,.6)

Upper arm length: 1
Lower arm length: 1
Shoulder: 28.64788975654116
Elbow 34.37746770784939
Horizonal: 1.33117868331595
Vertical: 1.3706328986656384
Bisection
Shoulder interval: [ 28.64772187614286 28.6480600699601 ]
Approximate  angle 28.647890973051478  degrees has been obtained in 18 steps
g(d)g(e)<0 not satisfied

Fixed-point
Approximate angle of shoulder: 28.647877300455534  degrees
Approximate angle of elbow: 88.32732933016675  degrees
Elbow-down configuation: Shoulder at  116.97520663062237  degrees, and elbow at  268.3273293301668  degrees

Newton
Approximate angle of shoulder: 28.647889756541154 degrees
Approximate angle of elbow: 34.37746770688771 degrees
Elbow-down configuation: Shoulder at  63.02535746342886  degrees, and elbow at  214.3774677068877  degrees



In [85]:
def checkMethodsRandom():
    checkMethods(random.uniform(0,10),random.uniform(0,10),random.uniform(0,2*math.pi),random.uniform(0,2*math.pi))

checkMethodsRandom()

Upper arm length: 3.8192622756454595
Lower arm length: 9.68587299797581
Shoulder: 25.434477731887085
Elbow 328.8388000643567
Horizonal: 13.08662014693439
Vertical: 0.6737981190304839
Bisection
Shoulder interval: [ 25.434182867392902 25.43448090792422 ]
Approximate  angle 25.43433188765856  degrees has been obtained in 21 steps
Elbow interval: [ 160.29206303344793 160.2925408041276 ]
Approximate  angle 160.29230191878776  degrees has been obtained in 20 steps
Elbow-down configuation: Shoulder at  84.21772981271107  degrees, and elbow at  340.29230191878776  degrees

Fixed-point
The first fixed point iteration diverges

Newton
Approximate angle of shoulder: 340.4603565255355 degrees
Approximate angle of elbow: 31.161199935643545 degrees
Elbow-down configuation could not be found.


In [87]:
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()

checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()

checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()

checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()

checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()
checkMethodsRandom()



Upper arm length: 7.716559889809456
Lower arm length: 1.1092635786612481
Shoulder: 327.18251747287707
Elbow 148.14187286982826
Horizonal: 6.010527382217635
Vertical: -3.179438843717799
Bisection
f(a)f(b)<0 not satisfied

Fixed-point
The first fixed point iteration diverges

Newton
Approximate angle of shoulder: 327.1825174728766 degrees
Approximate angle of elbow: 97.49309218442373 degrees
Elbow-down configuation: Shoulder at  335.37641421276044  degrees, and elbow at  277.49309218442374  degrees

Upper arm length: 0.05032387243931957
Lower arm length: 8.298350808261114
Shoulder: 79.54280854238843
Elbow 300.8846922745507
Horizonal: 7.785639307411071
Vertical: 2.9457940623727583
Bisection
Shoulder interval: [ 79.54250272529951 79.54298719405561 ]
Approximate  angle 79.54274495967756  degrees has been obtained in 20 steps
g(d)g(e)<0 not satisfied

Fixed-point
Approximate angle of shoulder: 321.9065868143739  degrees
The second fixed point iteration diverges

Newton
Approximate angle of s